 # Multi Query Retrievers
 This tutorial demonstrates how to build and use multi-query retrievers with LangChain.
 We'll start by loading documents, splitting them into chunks, and using embeddings
 to enable similarity search and retrieval.

In [3]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print
import uuid



 ## Step 1: Load Documents
 Here, we load the documents using `TextLoader`. You can replace the file paths
 with your own document paths.

In [4]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())



 ## Step 2: Split Documents into Chunks
 Using `RecursiveCharacterTextSplitter`, we split documents into smaller chunks
 for more efficient processing and retrieval.

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)



 ## Step 3: Initialize the Vectorstore
 The vectorstore indexes the document chunks for similarity search. We use the `Chroma`
 library with embeddings from Ollama.

In [6]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OllamaEmbeddings(model='snowflake-arctic-embed:33m')
)



 ## Step 4: Create Smaller Chunks
 For fine-grained retrieval, we split the documents further into smaller chunks
 and associate metadata for linking with original documents.

In [7]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

# Initialize storage for parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# Create the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

# Split into smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)



 ## Step 5: Add Documents to Vectorstore
 Add the smaller chunks and their metadata to the vectorstore for similarity search.

In [17]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))
print(retriever)

MultiVectorRetriever(
    vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x12c5c8290>,
    byte_store=<langchain_core.stores.InMemoryByteStore object at 0x12c3f9bd0>,
    docstore=<langchain.storage.encoder_backed.EncoderBackedStore object at 0x12c3860d0>,
    search_kwargs={}
)



 ## Step 6: Perform a Similarity Search
 Use the retriever to find documents similar to the query "LangChain".

In [9]:
print(retriever.vectorstore.similarity_search("LangChain")[0])

Document(
    id='defc5659-e538-472e-9aa5-abe17938a388',
    metadata={'doc_id': '9c4e0d26-1836-4056-8643-6ea0f58bf49b', 'source': 'data/langchain.md'},
    page_content='LangChain provides a consistent interface for working with chat models from different providers 
while offering additional features for monitoring, debugging, and optimizing the performance of applications that 
use LLMs.'
)



 ## Step 7: Multi-Modal Retrieval
 Modify the search type to use Maximal Marginal Relevance (MMR) for diverse results.

In [10]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr
retriever.invoke("LangChain")

[Document(metadata={'source': 'data/langchain.md'}, page_content='Title: Chat models | 🦜️🔗 LangChain\n\nURL Source: https://python.langchain.com/docs/concepts/chat_models/\n\nMarkdown Content:\nOverview[\u200b](https://python.langchain.com/docs/concepts/chat_models/#overview "Direct link to Overview")\n-------------------------------------------------------------------------------------------------------\n\nLarge Language Models (LLMs) are advanced machine learning models that excel in a wide range of language-related tasks such as text generation, translation, summarization, question answering, and more, without needing task-specific fine tuning for every scenario.\n\nModern LLMs are typically accessed through a chat model interface that takes a list of [messages](https://python.langchain.com/docs/concepts/messages/) as input and returns a [message](https://python.langchain.com/docs/concepts/messages/) as output.\n\nThe newest generation of chat models offer additional capabilities:\n



 ## Step 8: Summarize Documents
 Associate summaries with documents using a language model.

In [11]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# Initialize LLM
llm = ChatOllama(model='llama3.2:1b')

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

# Summarize documents in parallel
summaries = chain.batch(docs, {"max_concurrency": 5})
print(summaries)

[
    "The document provides an overview of LangChain, a platform for interacting with large language models (LLMs). 
Here's a summary:\n\n**Overview**\n\nLangChain is a platform for working with LLMs through a chat model interface. 
It allows users to interact with chat models by sending input messages and receiving output messages.\n\n**Key 
Features**\n\n1. **Standard Streaming Interface**: LangChain supports standard streaming interfaces, which allow 
for efficient processing of multiple requests in batches.\n2. **Async Programming**: Chat models support async 
programming, making it easy to write concurrent code.\n3. **Optimized Batching**: Many chat models have optimized 
batching capabilities, allowing for more efficient use of resources.\n\n**Inputs and Outputs**\n\n1. **Input 
Messages**: Modern LLMs are typically accessed through a chat model interface that takes input messages as 
input.\n2. **Output Messages**: The output messages are sent back to the user in response to their input 
messages.\n\n**Standard Parameters**\n\nSome chat models have standardized parameters, including:\n\n1. 
**Temperature**: Many chat models allow users to control the temperature of their outputs.\n2. **Maximum Number of 
Tokens**: Some chat models allow users to specify a maximum number of tokens for their responses.\n3. **Maximum 
Time to Wait**: Some chat models allow users to specify a maximum time to wait for a response.\n\n**Key 
Methods**\n\n1. **Invoke**: The primary method for interacting with a chat model, which takes input messages and 
returns output messages.\n2. **Stream**: A method that allows for streaming the output of a chat model as it is 
generated.\n3. **Batch**: A method that allows for batching multiple requests together for more efficient 
processing.\n4. **Bind Tools**: A method that allows users to bind tools to their chat models.\n5. **With 
Structured Output**: A wrapper around the invoke method for models that natively support structured 
output.\n\nOverall, LangChain provides a flexible and efficient way to interact with LLMs through a chat model 
interface.",
    "The document provides an overview of how to configure and use multiple AI models in a chat application. Here's
a summary of the key points:\n\n**Standard Parameters**\n\n*   `model`: The name or identifier of the specific AI 
model you want to use.\n*   `temperature`: Controls the randomness of the model's output, with higher values making
responses more creative and lower values focused on deterministic answers.\n*   `timeout`: Sets the maximum time 
(in seconds) to wait for a response from the model before canceling the request.\n*   `max_tokens`: Limits the 
total number of tokens in the response, controlling how long the output can be.\n*   `stop`: Specifies stop 
sequences that indicate when the model should stop generating tokens.\n*   `max_retries`: Sets the maximum number 
of attempts to resend a request if it fails due to issues like network timeouts or rate limits.\n*   `api_key`: The
API key required for authenticating with the model provider.\n*   `base_url`: The URL of the API endpoint where 
requests are sent.\n*   `rate_limiter`: An optional parameter that controls the rate at which requests are made to 
the model provider.\n\n**Important Notes**\n\n*   Standard parameters only apply to model providers that expose 
these parameters.\n*   Standard parameters are currently not enforced on integrations with models in 
`langchain-community`.\n*   Other parameters specific to each integration should be found in their respective API 
references.\n\n**Other Parameters and Features**\n\n*   Chat models can call tools to perform tasks such as 
fetching data from a database or running custom code.\n*   Structured outputs are available, allowing for 
information extraction tasks.\n*   Multimodality allows chat models to process other types of data, such as images,
audio, and video.\n*   Context windows refer to the maximum size of the input s



 ## Step 9: Add Summaries to Vectorstore
 Store the summaries in the vectorstore for enhanced retrieval capabilities.

In [12]:
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
store = InMemoryByteStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))



 ## Step 10: Retrieve Summaries
 Perform a similarity search on the summaries to get the most relevant results.

In [16]:
sub_docs = retriever.vectorstore.similarity_search("LangChain")
print("Sub Docs:\n",sub_docs[0])

retrieved_docs = retriever.invoke("LangChain")
print("Retrieved Docs:\n",retrieved_docs[0])
# len(retrieved_docs[0].page_content)

Sub Docs:

Document(
    id='c7bcae43-1331-4508-8117-3363375e2125',
    metadata={'doc_id': '9c4e0d26-1836-4056-8643-6ea0f58bf49b'},
    page_content="The document provides an overview of LangChain, a platform for interacting with large language 
models (LLMs). Here's a summary:\n\n**Overview**\n\nLangChain is a platform for working with LLMs through a chat 
model interface. It allows users to interact with chat models by sending input messages and receiving output 
messages.\n\n**Key Features**\n\n1. **Standard Streaming Interface**: LangChain supports standard streaming 
interfaces, which allow for efficient processing of multiple requests in batches.\n2. **Async Programming**: Chat 
models support async programming, making it easy to write concurrent code.\n3. **Optimized Batching**: Many chat 
models have optimized batching capabilities, allowing for more efficient use of resources.\n\n**Inputs and 
Outputs**\n\n1. **Input Messages**: Modern LLMs are typically accessed through a chat model interface that takes 
input messages as input.\n2. **Output Messages**: The output messages are sent back to the user in response to 
their input messages.\n\n**Standard Parameters**\n\nSome chat models have standardized parameters, including:\n\n1.
**Temperature**: Many chat models allow users to control the temperature of their outputs.\n2. **Maximum Number of 
Tokens**: Some chat models allow users to specify a maximum number of tokens for their responses.\n3. **Maximum 
Time to Wait**: Some chat models allow users to specify a maximum time to wait for a response.\n\n**Key 
Methods**\n\n1. **Invoke**: The primary method for interacting with a chat model, which takes input messages and 
returns output messages.\n2. **Stream**: A method that allows for streaming the output of a chat model as it is 
generated.\n3. **Batch**: A method that allows for batching multiple requests together for more efficient 
processing.\n4. **Bind Tools**: A method that allows users to bind tools to their chat models.\n5. **With 
Structured Output**: A wrapper around the invoke method for models that natively support structured 
output.\n\nOverall, LangChain provides a flexible and efficient way to interact with LLMs through a chat model 
interface."
)

Retrieved Docs:

Document(
    metadata={'source': 'data/langchain.md'},
    page_content='Title: Chat models | 🦜️🔗 LangChain\n\nURL Source: 
https://python.langchain.com/docs/concepts/chat_models/\n\nMarkdown 
Content:\nOverview[\u200b](https://python.langchain.com/docs/concepts/chat_models/#overview "Direct link to 
Overview")\n-------------------------------------------------------------------------------------------------------
\n\nLarge Language Models (LLMs) are advanced machine learning models that excel in a wide range of 
language-related tasks such as text generation, translation, summarization, question answering, and more, without 
needing task-specific fine tuning for every scenario.\n\nModern LLMs are typically accessed through a chat model 
interface that takes a list of [messages](https://python.langchain.com/docs/concepts/messages/) as input and 
returns a [message](https://python.langchain.com/docs/concepts/messages/) as output.\n\nThe newest generation of 
chat models offer additional capabilities:\n\n*   [Tool 
calling](https://python.langchain.com/docs/concepts/tool_calling/): Many popular chat models offer a native [tool 
calling](https://python.langchain.com/docs/concepts/tool_calling/) API. This API allows developers to build rich 
applications that enable LLMs to interact with external services, APIs, and databases. Tool calling can also be 
used to extract structured information from unstructured data and perform various other tasks.\n*   [Structured 
output](https://python.langchain.com/docs/concepts/structured_outputs/): A technique to make a chat model respond 
in a structured format, such as JSON that matches a given schema.\n*   
[Multimodality](https://python.langchain.com/docs/concepts/multimodality/): The ability to work with data other 
than text; for example, images, audio, and 
video.\n\nFeatures[\u200b](https://python.langchain.com/docs/concepts/chat_models/#features "Direct link to 
Features")\n-------------------------------------------------------------------------------------------------------
\n\nLangChain provides a consistent interface for working with chat models from different providers while offering 
additional features for monitoring, debugging, and optimizing the performance of applications that use LLMs.\n\n*  
Integrations with many chat model providers (e.g., Anthropic, OpenAI, Ollama, Microsoft Azure, Google Vertex, 
Amazon Bedrock, Hugging Face, Cohere, Groq). Please see [chat model 
integrations](https://python.langchain.com/docs/integrations/chat/) for an up-to-date list of supported models.\n* 
Use either LangChain\'s [messages](https://python.langchain.com/docs/concepts/messages/) format or OpenAI 
format.\n*   Standard [tool calling API](https://python.langchain.com/docs/concepts/tool_calling/): standard 
interface for binding tools to models, accessing tool call requests made by models, and sending tool results back 
to the model.\n*   Standard API for [structuring 
outputs](https://python.langchain.com/docs/concepts/structured_outputs/#structured-output-method) via the 
`with_structured_output` method.\n*   Provides support for [async 
programming](https://python.langchain.com/docs/concepts/async/), [efficient 
batching](https://python.langchain.com/docs/concepts/runnables/#optimized-parallel-execution-batch), [a rich 
streaming API](https://python.langchain.com/docs/concepts/streaming/).\n*   Integration with 
[LangSmith](https://docs.smith.langchain.com/) for monitoring and debugging production-grade applications based on 
LLMs.\n*   Additional features like standardized [token 
usage](https://python.langchain.com/docs/concepts/messages/#aimessage), [rate 
limiting](https://python.langchain.com/docs/concepts/chat_models/#rate-limiting), 
[caching](https://python.langchain.com/docs/concepts/chat_models/#caching) and 
more.\n\nIntegrations[\u200b](https://python.langchain.com/docs/concepts/chat_models/#integrations "Direct link to 
Integrations")\n-----------------

In [14]:
# loop through the retrieved documents and print the content of each
for doc in retrieved_docs:
	print(doc.page_content)

Title: Chat models | 🦜️🔗 LangChain

URL Source: https://python.langchain.com/docs/concepts/chat_models/

Markdown Content:
Overview[​](https://python.langchain.com/docs/concepts/chat_models/#overview "Direct link to Overview")
-------------------------------------------------------------------------------------------------------

Large Language Models (LLMs) are advanced machine learning models that excel in a wide range of language-related 
tasks such as text generation, translation, summarization, question answering, and more, without needing 
task-specific fine tuning for every scenario.

Modern LLMs are typically accessed through a chat model interface that takes a list of 
(https://python.langchain.com/docs/concepts/messages/) as input and returns a 
(https://python.langchain.com/docs/concepts/messages/) as output.

The newest generation of chat models offer additional capabilities:

*   [Tool calling](https://python.langchain.com/docs/concepts/tool_calling/): Many popular chat models offer a 
native (https://python.langchain.com/docs/concepts/tool_calling/) API. This API allows developers to build rich 
applications that enable LLMs to interact with external services, APIs, and databases. Tool calling can also be 
used to extract structured information from unstructured data and perform various other tasks.
*   [Structured output](https://python.langchain.com/docs/concepts/structured_outputs/): A technique to make a chat
model respond in a structured format, such as JSON that matches a given schema.
*   [Multimodality](https://python.langchain.com/docs/concepts/multimodality/): The ability to work with data other
than text; for example, images, audio, and video.

Features[​](https://python.langchain.com/docs/concepts/chat_models/#features "Direct link to Features")
-------------------------------------------------------------------------------------------------------

LangChain provides a consistent interface for working with chat models from different providers while offering 
additional features for monitoring, debugging, and optimizing the performance of applications that use LLMs.

*   Integrations with many chat model providers (e.g., Anthropic, OpenAI, Ollama, Microsoft Azure, Google Vertex, 
Amazon Bedrock, Hugging Face, Cohere, Groq). Please see (https://python.langchain.com/docs/integrations/chat/) for 
an up-to-date list of supported models.
*   Use either LangChain's (https://python.langchain.com/docs/concepts/messages/) format or OpenAI format.
*   Standard (https://python.langchain.com/docs/concepts/tool_calling/): standard interface for binding tools to 
models, accessing tool call requests made by models, and sending tool results back to the model.
*   Standard API for (https://python.langchain.com/docs/concepts/structured_outputs/#structured-output-method) via 
the `with_structured_output` method.
*   Provides support for (https://python.langchain.com/docs/concepts/async/), 
(https://python.langchain.com/docs/concepts/runnables/#optimized-parallel-execution-batch), 
(https://python.langchain.com/docs/concepts/streaming/).
*   Integration with [LangSmith](https://docs.smith.langchain.com/) for monitoring and debugging production-grade 
applications based on LLMs.
*   Additional features like standardized (https://python.langchain.com/docs/concepts/messages/#aimessage), 
(https://python.langchain.com/docs/concepts/chat_models/#rate-limiting), 
(https://python.langchain.com/docs/concepts/chat_models/#caching) and more.

Integrations[​](https://python.langchain.com/docs/concepts/chat_models/#integrations "Direct link to Integrations")
-------------------------------------------------------------------------------------------------------------------

LangChain has many chat model integrations that allow you to use a wide variety of models from different providers.

These integrations are one of two types:

1.  **Official models**: These are models that are officially supported by LangChain and/or model provide

Vector store support for metadata filtering is typically dependent on the underlying vector store implementation.

Here is example usage with 
[Pinecone](https://python.langchain.com/docs/integrations/vectorstores/pinecone/#query-directly), showing that we 
filter for all documents that have the metadata key `source` with value `tweet`.

```
vectorstore.similarity_search(    "LangChain provides abstractions to make working with LLMs easy",    k=2,    
filter={"source": "tweet"},)
```

Advanced search and retrieval 
techniques[​](https://python.langchain.com/docs/concepts/vectorstores/#advanced-search-and-retrieval-techniques 
"Direct link to Advanced search and retrieval techniques")
-------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------

While algorithms like HNSW provide the foundation for efficient similarity search in many cases, additional 
techniques can be employed to improve search quality and diversity. For example, 
(https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/mmr/) is a re-ranking algorithm 
used to diversify search results, which is applied after the initial similarity search to ensure a more diverse set
of results. As a second example, some 
(https://python.langchain.com/docs/integrations/retrievers/pinecone_hybrid_search/) offer built-in 
(https://docs.pinecone.io/guides/data/understanding-hybrid-search) to combine keyword and semantic similarity 
search, which marries the benefits of both approaches. At the moment, there is no unified way to perform hybrid 
search using LangChain vectorstores, but it is generally exposed as a keyword argument that is passed in with 
`similarity_search`. See this (https://python.langchain.com/docs/how_to/hybrid/) for more details.

| Name                                                                                                             
| When to use                                           | Description                                              
|
| 
-------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------
--------- | ----------------------------------------------------- | 
-------------------------------------------------------------------------------------------------------------------
------------------------- |
| [Hybrid search](https://python.langchain.com/docs/integrations/retrievers/pinecone_hybrid_search/)               
| When combining keyword-based and semantic similarity. | Hybrid search combines keyword and semantic similarity, 
marrying the benefits of both approaches. [Paper](https://arxiv.org/abs/2210.11934). |
| [Maximal Marginal Relevance 
(MMR)](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVec
torStore.html#langchain_pinecone.vectorstores.PineconeVectorStore.max_marginal_relevance_search) | When needing to 
diversify search results.             | MMR attempts to diversify the results of a search to avoid returning 
similar and redundant documents.                                        |

Title: Vector stores | 🦜️🔗 LangChain

URL Source: https://python.langchain.com/docs/concepts/vectorstores/

Markdown Content:
Note

This conceptual overview focuses on text-based indexing and retrieval for simplicity. However, embedding models can
be (https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-multimodal-embeddings) and vector stores 
can be used to store and retrieve a variety of data types beyond text.

Overview[​](https://python.langchain.com/docs/concepts/vectorstores/#overview "Direct link to Overview")
--------------------------------------------------------------------------------------------------------

Vector stores are specialized data stores that enable indexing and retrieving information based on vector 
representations.

These vectors, called (https://python.langchain.com/docs/concepts/embedding_models/), capture the semantic meaning 
of data that has been embedded.

Vector stores are frequently used to search over unstructured data, such as text, images, and audio, to retrieve 
relevant information based on semantic similarity rather than exact keyword matches.

![Image 5: Vector 
stores](https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png)

Integrations[​](https://python.langchain.com/docs/concepts/vectorstores/#integrations "Direct link to Integrations")
-------------------------------------------------------------------------------------------------------------------
-

LangChain has a large number of vectorstore integrations, allowing users to easily switch between different 
vectorstore implementations.

Please see the (https://python.langchain.com/docs/integrations/vectorstores/).

Interface[​](https://python.langchain.com/docs/concepts/vectorstores/#interface "Direct link to Interface")
-----------------------------------------------------------------------------------------------------------

LangChain provides a standard interface for working with vector stores, allowing users to easily switch between 
different vectorstore implementations.

The interface consists of basic methods for writing, deleting and searching for documents in the vector store.

The key methods are:

*   `add_documents`: Add a list of texts to the vector store.
*   `delete_documents`: Delete a list of documents from the vector store.
*   `similarity_search`: Search for similar documents to a given query.

Initialization[​](https://python.langchain.com/docs/concepts/vectorstores/#initialization "Direct link to 
Initialization")
-------------------------------------------------------------------------------------------------------------------
-------

Most vectors in LangChain accept an embedding model as an argument when initializing the vector store.

We will use LangChain's 
[InMemoryVectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.in_m
emory.InMemoryVectorStore.html) implementation to illustrate the API.

```
from langchain_core.vectorstores import InMemoryVectorStore# Initialize with an embedding modelvector_store = 
InMemoryVectorStore(embedding=SomeEmbeddingModel())
```

Adding documents[​](https://python.langchain.com/docs/concepts/vectorstores/#adding-documents "Direct link to Adding
documents")
-------------------------------------------------------------------------------------------------------------------
-------------

To add documents, use the `add_documents` method.

This API works with a list of 
[Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) 
objects. `Document` objects all have `page_content` and `metadata` attributes, making them a universal way to store
unstructured text and associated metadata.

```
from langchain_core.documents import Documentdocument_1 = Document(    page_content="I had chocalate chip pancakes 
and scrambled eggs for breakfast this morning.",    metadata={"source": "tweet"},)document_2 = Document(    
page_co

The reason is that getting a cache hit is unlikely after the first or second interaction in a conversation if 
relying on caching the **exact** inputs into the model. For example, how likely do you think that multiple 
conversations start with the exact same message? What about the exact same three messages?

An alternative approach is to use semantic caching, where you cache responses based on the meaning of the input 
rather than the exact input itself. This can be effective in some situations, but not in others.

A semantic cache introduces a dependency on another model on the critical path of your application (e.g., the 
semantic cache may rely on an (https://python.langchain.com/docs/concepts/embedding_models/) to convert text to a 
vector representation), and it's not guaranteed to capture the meaning of the input accurately.

However, there might be situations where caching chat model responses is beneficial. For example, if you have a 
chat model that is used to answer frequently asked questions, caching responses can help reduce the load on the 
model provider, costs, and improve response times.

Please see the (https://python.langchain.com/docs/how_to/chat_model_caching/) guide for more details.

*   How-to guides on using chat models: (https://python.langchain.com/docs/how_to/#chat-models).
*   List of supported chat models: (https://python.langchain.com/docs/integrations/chat/).

### Conceptual guides[​](https://python.langchain.com/docs/concepts/chat_models/#conceptual-guides "Direct link to 
Conceptual guides")

*   [Messages](https://python.langchain.com/docs/concepts/messages/)
*   [Tool calling](https://python.langchain.com/docs/concepts/tool_calling/)
*   [Multimodality](https://python.langchain.com/docs/concepts/multimodality/)
*   [Structured outputs](https://python.langchain.com/docs/concepts/structured_outputs/)